In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

### Connections Scraper

In [32]:
def browser_wait(driver, element, locator=By.XPATH, timeout=10):
    return WebDriverWait(driver, timeout).until(
        EC.visibility_of_element_located((locator, element))
    )

def scroll(driver, start=0, end='document.body.scrollHeight'):
    driver.execute_script(f'window.scrollTo({start}, {end});')    

In [5]:
# env
USERNAME = 'fahri.lafa@gmail.com'
PASSWORD = '1qw23er4'

In [15]:
# specify chrome options
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--disable-gpu')
options.add_argument('--disable-extensions')
options.add_argument('--headless')
browser = webdriver.Chrome(options=options)

# open the site
print('Opening site...')
browser.get('https://linkedin.com')

# move to login page
try:    
    login_btn = browser_wait(browser, '/html/body/nav/div/a[2]')
    login_btn.click()
except:
    print('Cannot find login button!')

# fill credentials
print('Attempting to login...')
try:
    email_field = browser_wait(browser, '#username', locator=By.CSS_SELECTOR)
    pwd_field = browser_wait(browser, '#password', locator=By.CSS_SELECTOR)
    # insert email password
    email_field.send_keys(USERNAME)
    pwd_field.send_keys(PASSWORD)
    # submit
    time.sleep(1.5)
    pwd_field.send_keys(Keys.RETURN)
    print('Submit login...')
except:
    print('Cannot find email or passowrd field!')

# go to My Network page
print('Move to `My Network` page')
browser.get('https://www.linkedin.com/mynetwork/invite-connect/connections/')

# scroll to bottom
last_height = browser.execute_script('return document.body.scrollHeight')
wait_scroll = 5
while True:
    print('Scrolling...')
    # scroll until the last record
    scroll(browser)
    print('Wait 5s...')
    time.sleep(5)
    new_height = browser.execute_script('return document.body.scrollHeight')
    # if scroll reach the end
    if new_height == last_height:
        try:
            # look for load_more button
            print('Looking for load more button...')
            # load_more = browser_wait(browser, '/html/body/div[7]/div[3]/div/div/div/div/div[2]/div/div/main/div/section/div[2]/div[2]/div/button')
            load_more = browser_wait(browser, 'scaffold-finite-scroll__load-button', locator=By.CLASS_NAME)
            load_more.click()
            print('Wait 5s...')
            time.sleep(5)
            # scroll again
            print('Scrolling...')
            scroll(browser)
        except:
            # no more data
            print('Load more button not found! It seems we are reached the end')
            break
    # update last height
    print('Update last height')
    last_height = new_height
            
# get raw html
print('Get output...')
my_networks = BeautifulSoup(browser.page_source.encode('utf-8'), 'html')
my_networks.prettify()

# write html to file
print('Write output...')
with open('my_networks.html', 'w', encoding='utf-8') as w:
    w.write(str(my_networks))

# # anaylzing html tags
# print('Analyzing html tags...')

# quit browser
print('Done. Quitting...')
time.sleep(5)

browser.close()

Opening site...
Attempting to login...
Submit login...
Move to `My Network` page
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Looking for load more button...
Wait 5s...
Scrolling...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scrolling...
Wait 5s...
Update last height
Scr

In [17]:
containers = my_networks.findAll('li', {'class': 'mn-connection-card artdeco-list'})
len(containers)

839

In [18]:
# parsing profiles list
profiles = []

for c in containers:
    # data model
    m_profile = {
        'name': None,
        'occupation': None,
        'profile_link': 'https://linkedin.com{}',
        'connected_at': None
    }
    m_profile['name'] = c.find('span', {'class': 'mn-connection-card__name'}).text.strip()
    m_profile['occupation'] = c.find('span', {'class': 'mn-connection-card__occupation'}).text.strip()
    m_profile['profile_link'] = m_profile['profile_link'].format(c.find('a', {'class': 'mn-connection-card__link'})['href'])
    m_profile['connected_at'] = c.find('time').text.strip().split('\n')[-1].strip()
    profiles.append(m_profile)    

df_profile = pd.DataFrame(profiles)
df_profile.head()

,name,occupation,profile_link,connected_at
0,"Sie, Evan Setiawan",Class Coordinator - Sekolah Data PACMANN,https://linkedin.com/in/sie-evan-setiawan/,1 day ago
1,Erika Dwi Puspitasari,Student at Airlangga University,https://linkedin.com/in/erika-dwi-puspitasari-...,2 days ago
2,Ekaterina Shlenova,IT Recruiter in Yandex,https://linkedin.com/in/ekaterina-shlenova-714...,3 days ago
3,eef hikmat,Student at sekolah tinggi ilmu ekonomi 66,https://linkedin.com/in/eef-hikmat-13332a1a6/,1 week ago
4,Amna Maharani,IT Student at Fachhochschule Kiel,https://linkedin.com/in/amaharani/,1 week ago


### Profile Scraper

In [10]:
df_profile.profile_link

0             https://linkedin.com/in/sie-evan-setiawan/
1      https://linkedin.com/in/erika-dwi-puspitasari-...
2      https://linkedin.com/in/ekaterina-shlenova-714...
3          https://linkedin.com/in/eef-hikmat-13332a1a6/
4                     https://linkedin.com/in/amaharani/
                             ...                        
834                  https://linkedin.com/in/nabilah-iz/
835    https://linkedin.com/in/zikra-halim-arrassy-a4...
836                       https://linkedin.com/in/magaz/
837    https://linkedin.com/in/soegianto-soelistiono-...
838    https://linkedin.com/in/cendra-devayana-putra-...
Name: profile_link, Length: 839, dtype: object

In [2]:
from actions import Actions
from credentials import Credentials
import constant

c = Credentials()
c.get_uid()

1

In [ ]:
# specify chrome options
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--disable-gpu')
options.add_argument('--disable-extensions')
# options.add_argument('--headless')
browser = webdriver.Chrome(options=options)

actions = Actions(browser, c)
actions.do_login()

In [119]:
browser.get('https://linkedin.com/in/sie-evan-setiawan/')
# link = browser_wait(browser, 'link-without-visited-state:nth-child(2)', locator=By.CLASS_NAME, timeout=120)
# WebDriverWait(browser, 10)
# link = browser.find_element(By.XPATH, '/html/body/div[6]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/ul/li/a')
# link = WebDriverWait(browser, 30).until(
#         EC.presence_of_element_located((By.CSS_SELECTOR, 'li.text-body-small:nth-child(2)'))
#     )
link = browser_wait(browser, 'li.text-body-small:nth-child(2)', By.CSS_SELECTOR, 30)
print(link)

<selenium.webdriver.remote.webelement.WebElement (session="82bb0f6927ca24d3dc34122684cb4d0a", element="faf6ec61-aeb2-4876-8e4c-f0b0ad1af7e1")>


In [72]:
link.click()

In [75]:
filter_btn = browser_wait(browser, 'div.search-reusables__filter-trigger-and-dropdown:nth-child(1)', By.CSS_SELECTOR, timeout=30)
filter_btn.click()

first_circle = browser_wait(browser, 'li.search-reusables__collection-values-item:nth-child(1) > label', By.CSS_SELECTOR, timeout=30)
first_circle.click()
third_circle = browser_wait(browser, 'li.search-reusables__collection-values-item:nth-child(3) > label', By.CSS_SELECTOR, timeout=30)
third_circle.click()

In [67]:
submit = browser_wait(browser, '/html/body/div[7]/div[3]/div[2]/section/div/nav/div/ul/li[3]/div/div/div/div[1]/div/form/fieldset/div[2]/button[2]')
submit.click()

In [68]:
next = browser_wait(browser, '/html/body/div[7]/div[3]/div[2]/div/div[1]/main/div/div/div[3]/div/div/button[2]')
next

<selenium.webdriver.remote.webelement.WebElement (session="3bec0805c7b54e7b05113556ffb54bfa", element="8969d98c-084f-44f7-8ecf-4962b03821fa")>

In [122]:
WebDriverWait(browser, 30).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.artdeco-pagination__button--next')))
                                        #       '/html/body/div[7]/div[3]/div[2]/div/div[1]/main/div/div/div[3]/div/div/button[2]'
                                        #       '/html/body/div[7]/div[3]/div[2]/div/div[1]/main/div/div/div/div[2]/div/button[2]'
        # EC.element_to_be_clickable((By.XPATH, '/html/body/div[7]/div[3]/div[2]/div/div[1]/main/div/div/div[3]/div/div/button[1]')))

<selenium.webdriver.remote.webelement.WebElement (session="82bb0f6927ca24d3dc34122684cb4d0a", element="1b0c618d-41bb-4be9-aa41-0ddf2753ea8d")>

In [ ]:
# scrap item
connections = BeautifulSoup(browser.page_source.encode('utf-8'), 'html')
connections.prettify()

In [80]:
containers = connections.findAll('div', {'class': 'entity-result__item'})
len(containers)

10

In [109]:
for c in containers:
    heading = c.find('span', {'class': 'entity-result__title-text t-16'})    
    # data properties
    name = heading.find('span', {'dir': 'ltr'}).span.text
    circle_level = heading.find('span', {'class': 'image-text-lockup__text entity-result__badge-text'}).span.text.split()[-1][0]
    profile_link = heading.a['href']
    occupation = c.find('div', {'class': 'entity-result__primary-subtitle t-14 t-black t-normal'}).text.strip()
    
    break

Ladinda Tasya


In [4]:
import threading
from queue import Queue
import time

In [5]:
qlist = []

for i in range(5):
    # specify chrome options
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    options.add_argument('--disable-gpu')
    options.add_argument('--disable-extensions')
    # options.add_argument('--headless')
    browser = webdriver.Chrome(options=options)
    
    actions = Actions(browser, c)
    actions.do_login()

    qlist.append(browser)

len(qlist)

WARNING - li_at value is None.
Attempting to login...
WARNING - li_at value is None.
Attempting to login...
WARNING - li_at value is None.
Attempting to login...
WARNING - li_at value is None.
Attempting to login...
WARNING - li_at value is None.
Attempting to login...


5

In [7]:
from objectpool import ObjectPool
pool = ObjectPool.get_instance()
pool.set_resource(qlist)
pool.get_total_resource()

5

In [15]:
links = [
    'https://linkedin.com/in/sie-evan-setiawan/',
    'https://linkedin.com/in/erika-dwi-puspitasari-965b7318b/',
    'https://linkedin.com/in/ekaterina-shlenova-714504189/',
    'https://linkedin.com/in/eef-hikmat-13332a1a6/',
    'https://linkedin.com/in/amaharani/',
    'https://linkedin.com/in/nemat-allah-aloush-60887a1a9/',
    'https://linkedin.com/in/rayhanozzy/',
    'https://linkedin.com/in/reynaldipriandika/',
    'https://linkedin.com/in/faisal-ichsan/',
    'https://linkedin.com/in/sisca-fahrudha-retno-agustina-119396183/',
    'https://linkedin.com/in/muhammad-abil-nurjani/',
    'https://linkedin.com/in/mishrag37/',
    'https://linkedin.com/in/rahmatri-mardiko-56720690/',
    'https://linkedin.com/in/bintang-muhammad-326827140/',
    'https://linkedin.com/in/hanifizzudinr/',
    'https://linkedin.com/in/frdaamalia/',
    'https://linkedin.com/in/andrey-kiselevich-368165232/',
    'https://linkedin.com/in/masetarahma/',
    'https://linkedin.com/in/ika-sulistiorini-a3032a152/',
    'https://linkedin.com/in/cerelia-devina-tantia-vinci-ninggalih-1949571b6/',
    'https://linkedin.com/in/ilhamnurakbar/',
    'https://linkedin.com/in/wahyu-nur-abidin-b20b26222/',
    'https://linkedin.com/in/houssam-hammoudi/',
    'https://linkedin.com/in/vladislav-vasiljev-a3a22021a/',
    'https://linkedin.com/in/ayu-anggraini30/'
]

In [16]:
lock_db = threading.Lock()

def dumper_thread(profile_link, op, thread_name):
    while True:
        print(f'{thread_name} borrowing object...')
        driver = op.borrow_resource()
        print(f'{thread_name} opening link...')
        driver.get(profile_link)
        print(f'{thread_name} dumping html...')
        # TODO: dump html
        with lock_db:
            print(f'{thread_name} inserting to db...')
            time.sleep(10)
        print(f'{thread_name} returning resource...')
        op.return_resource(driver)
        print(f'{thread_name} done.')
        break

threads = []
for i, item in enumerate(links):
    thread_name = f'Thread-{i}'
    t = threading.Thread(target=dumper_thread, args=(item, pool, thread_name))
    t.name = thread_name
    threads.append(t)

In [19]:
max_thread_exec = 5
curr_thread = 0

for _ in range(max_thread_exec):
    threads[curr_thread].start()
    curr_thread += 1

Thread-24 borrowing object...
Thread-24 opening link...


RuntimeError: threads can only be started once

Thread-24 dumping html...
Thread-24 inserting to db...
Thread-24 returning resource...
Thread-24 done.


In [18]:
curr_thread

5

In [39]:
insert_count = 0

def dumper_thread(queue):
    while True:        
        try:
            i, item = queue.get()
            thread_name = f'Thread-{i}'
            print(f'{thread_name} borrowing resource...')
            driver = pool.borrow_resource()
            print(f'{thread_name} opening link...')
            driver.get(item)
            print(f'{thread_name} dumping page...')                
            time.sleep(3)

            with lock_db:
                print(f'{thread_name} inserting to db...')
                time.sleep(10)

            print(f'{thread_name} done.')
            queue.task_done()
            
        except:
            raise SystemError('No free resource currently.')
    

for _ in range(5):    
    t = threading.Thread(target=dumper_thread, args=(qlinks))
    t.start()

Exception in thread Thread-74:
Traceback (most recent call last):
  File "c:\Users\fahrizain\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-75:
Traceback (most recent call last):
  File "c:\Users\fahrizain\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-76:
Traceback (most recent call last):
  File "c:\Users\fahrizain\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-77:
Traceback (most recent call last):
  File "c:\Users\fahrizain\Anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
        self.run()
  File "c:\Users\fahrizain\Anaconda3\lib\threading.py", line 870, in run
self.run()
  File "c:\Users\fahrizain\Anaconda3\lib\threading.py", line 870, in run
      File "c:\Users\fahrizain\Anaconda3\lib\threading.py", line 870, in run
Exception in thread Thread-78:
Traceback (most recent call last):
  File "c:\Users\fahrizain\Anaconda3\lib\threading.py", li

In [30]:
lock = threading.Lock()
count = 0

def thread_test(i):
    print(f'Thread-{i} attempting to update count...')
    with lock:
        global count
        count += 1
        print(f'Count was updated by thread-{i}!')
        time.sleep(10)

def threader():
    while True:
        i = q.get()
        print(f'Thread-{i} starting...')
        thread_test(i)
        time.sleep(3)
        print(f'Thread-{i} done.')
        q.task_done()

q = Queue()
for x in range(5):
    thread = threading.Thread(target=threader)

    # thread.daemon = True
    thread.start()

for job in range(10):
    q.put(job)

Thread-0 starting...
Thread-0 attempting to update count...
Count was updated by thread-0!
Thread-1 starting...
Thread-1 attempting to update count...
Thread-2 starting...
Thread-2 attempting to update count...
Thread-3 starting...
Thread-3 attempting to update count...
Thread-4 starting...
Thread-4 attempting to update count...
Count was updated by thread-1!
Thread-0 done.
Thread-5 starting...
Thread-5 attempting to update count...
Count was updated by thread-2!
Thread-1 done.
Thread-6 starting...
Thread-6 attempting to update count...
Count was updated by thread-3!
Thread-2 done.
Thread-7 starting...
Thread-7 attempting to update count...
Count was updated by thread-4!
Thread-3 done.
Thread-8 starting...
Thread-8 attempting to update count...
Count was updated by thread-5!
Thread-4 done.
Thread-9 starting...
Thread-9 attempting to update count...
Count was updated by thread-6!
Thread-5 done.
Count was updated by thread-7!
Thread-6 done.
Count was updated by thread-8!
Thread-7 done.
C

In [27]:
print(count)

10
